<a href="https://colab.research.google.com/github/arjun-2131/cow-breed/blob/master/cow_breedv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,BatchNormalization
from keras import backend as K
from keras.preprocessing import image
from keras import applications
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from keras.applications.xception import preprocess_input
import seaborn as sns
sns.set(color_codes=True)

In [0]:
batch_size=9
num_classes=4
epochs=10
img_width,img_height=256,256

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_data_dir="/content/drive/My Drive/cow/train"
validation_data_dir="/content/drive/My Drive/cow/valid"
test_data_dir="/content/drive/My Drive/cow/test"

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [0]:
  model=applications.xception.Xception(include_top=False,
                                      weights='imagenet',
                                      input_shape=(img_width,img_height,3))

In [0]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=BatchNormalization()(x)
x=Dense(1024,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
y_pred=Dense(4,activation='softmax')(x)

In [41]:
model_final=Model(inputs=model.input,outputs=y_pred)
model_final.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
for layers in model.layers[:126]:
  layers.trainable=False
for layer in model.layers[126:]:
  layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(
              rescale=1./255,
              shear_range=0.2,
              zoom_range=0.2,
              horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [44]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=test_datagen.flow_from_directory(validation_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

Found 329 images belonging to 4 classes.
Found 33 images belonging to 4 classes.
Found 33 images belonging to 4 classes.


In [0]:
model_final.compile(loss='categorical_crossentropy',
                   optimizer='nadam',
                   metrics=['accuracy'])   

In [47]:
top_model_weight_path='cow_Xception.h5'
checkpoint=ModelCheckpoint(filepath=top_model_weight_path,
                          verbose=1,
                            save_best_only=True,
                          monitor='val_acc')
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=5,verbose=1,mode='auto')

history=model_final.fit_generator(
              train_generator,
                steps_per_epoch=329//batch_size,
              epochs=epochs,
                validation_data=validation_generator,
                validation_steps=33//batch_size,
                callbacks=[checkpoint,early])

Epoch 1/10
36/36 [==============================] - 28s 788ms/step - loss: 1.6341 - acc: 0.7105 - val_loss: 1.2741 - val_acc: 0.7500

Epoch 00001: val_acc improved from -inf to 0.75000, saving model to cow_Xception.h5
Epoch 2/10
36/36 [==============================] - 9s 264ms/step - loss: 1.0103 - acc: 0.7877 - val_loss: 1.4600 - val_acc: 0.7917

Epoch 00002: val_acc improved from 0.75000 to 0.79167, saving model to cow_Xception.h5
Epoch 3/10
36/36 [==============================] - 11s 299ms/step - loss: 0.7265 - acc: 0.8401 - val_loss: 1.3476 - val_acc: 0.7083

Epoch 00003: val_acc did not improve from 0.79167
Epoch 4/10
36/36 [==============================] - 11s 303ms/step - loss: 0.7682 - acc: 0.8333 - val_loss: 0.5247 - val_acc: 0.7778

Epoch 00004: val_acc did not improve from 0.79167
Epoch 5/10
36/36 [==============================] - 11s 303ms/step - loss: 0.7450 - acc: 0.7938 - val_loss: 1.9819 - val_acc: 0.5000

Epoch 00005: val_acc did not improve from 0.79167
Epoch 6/10

In [0]:
scores,acc=model_final.evaluate_generator(test_generator,steps=12)

In [61]:
img=image.load_img("/content/drive/My Drive/cow/images.jpeg")
target_size=(256,256)
if img.size!=target_size:
  img=img.resize(target_size)
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
preds=model_final.predict(x)
print("probabilities for sindhi,ongole,kangrej,khillari:",preds[0])

probabilities for sindhi,ongole,kangrej,khillari: [3.3811171e-05 3.6326090e-03 9.9632120e-01 1.2321609e-05]


In [53]:
print (acc)

0.545454555388653


In [0]:
print(model_final.metrics_names)